In [106]:
import pandas as pd
import numpy as np
import random

In [107]:
df_d1 = pd.read_csv('/Users/steja/Downloads/University of Leeds/Semester 1/Programming for Data Science/Coursework 2 - GROUP PROJECT/Datasets/Swiggy_dataset.csv')
df_d1.head()

,type,id,name,uuid,city,area,avgRating,totalRatingsString,cuisines,costForTwoStrings,deliveryTime,minDeliveryTime,maxDeliveryTime,address,locality,unserviceable,veg,City
0,F,37933,Faasos - Wraps & Rolls,6fe9caf1-02a7-4e66-83bb-1b4ff296b683,Ahmedabad,Vastrapur,4.2,500+ ratings,['Combo' 'Snacks' 'Beverages' 'Desserts' 'Indi...,₹200 FOR TWO,44,44,44,SHOP NO 2 HOTEL SHAHI PALACE VASTRAPUR LAKE AH...,HOTEL SHAHI PALACE,False,False,NaN
1,F,81814,Burger King,10083576-d32d-4a0e-8a82-3236ef342a19,Ahmedabad,Ellisbridge,4.2,1000+ ratings,['American' 'Fast Food'],₹350 FOR TWO,33,33,33,SHOP # 5 GR FLR THIRD EYE 2PANCHAVATI CIRCLE O...,3rd Eye II Ellis Bridge CG Road,False,False,NaN
2,F,107046,Mahalaxmi Pav Bhaji,fbfe3bfa-03fd-4708-b913-06e2c9ee9639,Ahmedabad,Ellisbridge,4.3,500+ ratings,['North Indian'],₹200 FOR TWO,28,28,28,GF 9/10 Dev complexnear parimal charasta opp. ...,C G Road,False,True,NaN
3,F,328315,HIIT Meals,81ab4019-2445-4aef-aff2-04beccd641d9,Ahmedabad,Vishala,--,Too Few Ratings,['Fast Food' 'Combo' 'Snacks' 'North Indian' '...,₹150 FOR TWO,51,51,51,C11 Rajyash Rise Near Vishala Restaurant Vasna...,Vasna,False,False,NaN
4,F,108879,Jay Jalaram Parotha House,994e73e1-7c7c-4ad9-87e7-ecfecd4e36fa,Ahmedabad,Ellisbridge,3.9,100+ ratings,['North Indian'],₹250 FOR TWO,29,29,29,Capital Commercial Centre Near UCO Bank Ashram...,Ashram Road,False,True,NaN


In [108]:
df_d1.shape

(8691, 18)

In [109]:
# methods to clean the columns data and store in one format

def remove_list_like_structure(cuisine_list,dataFrame):
    
    for cuisine in cuisine_list:
        if '[' in cuisine:
            temp_cuisine = cuisine[1:-1]
            temp_cuisine_list = []
            quote_idx = [-1,-1]
            count = 0
            for i in range(len(temp_cuisine)):
                if temp_cuisine[i] == "'" and count <2:
                    quote_idx[count] = i
                    count += 1
                if count >1:
                    temp_cuisine_list.append(temp_cuisine[quote_idx[0]+1:quote_idx[1]])
                    count =0
            dataFrame.replace(cuisine,', '.join(temp_cuisine_list),inplace=True)

def count_occurrence_of_each_cuisine(cuisine_list):
    
    cuisine_occurrence_count = {}
    for cuisine in cuisine_list:
        for cuisine_name in cuisine.split(','):
            cuisine_name = cuisine_name.strip()
            if cuisine_name in cuisine_occurrence_count:
                cuisine_occurrence_count[cuisine_name] = cuisine_occurrence_count.get(cuisine_name) + 1
            else:
                cuisine_occurrence_count[cuisine_name] = 1
    return cuisine_occurrence_count

def replace_cuisine_value_with_most_ordered(dataFrame,cuisine_list,cuisine_occurrence_count,cuisine_col_name,id_col_name):
    modified_cuisine_col_value = {}
    for index,row in cuisine_list.iterrows():
        id = row[id_col_name]
        cusinie = row[cuisine_col_name]
        cusinie_list_temp = cusinie.split(",")
        cusinie_list_temp = sorted(cusinie_list_temp)
        cusinie_type = ''
        max_occurrance = -1
        for cuisine_name in cusinie_list_temp:
            cuisine_name = cuisine_name.strip()
            if cuisine_occurrence_count.get(cuisine_name) > max_occurrance:
                cusinie_type = cuisine_name
                max_occurrance = cuisine_occurrence_count.get(cuisine_name)
        #dataFrame.replace(cusinie,cusinie_type,inplace=True)
        modified_cuisine_col_value[id] = cusinie_type
    return modified_cuisine_col_value

def split_city_area(city_entry):
  city_entry_org = city_entry
  city_entry = city_entry.strip()
  for sep in [',', '&', '/']:
    if sep in city_entry and len(city_entry.split(sep)) == 2:
      parts = city_entry.split(sep)
      return pd.DataFrame({'city':city_entry_org,'area': [parts[0].strip()], 'new_city': [parts[1].strip()]})

def modify_rating_value(col_list):
    new_rating_dict = {}
    for rate in col_list:
        modified_rate_value = rate.split('/')[0]
        new_rating_dict[rate] = modified_rate_value
    return new_rating_dict

def replace_nan_values(dataFrame, column_name):
    value_counts = column_name.dropna().value_counts(normalize = True)
    return column_name.apply(lambda x: x if pd.notna(x) else np.random.choice(value_counts.index, p=value_counts.values))

    #fill_values = dataFrame[column_name].dropna().unique()
    #dataFrame[column_name] = dataFrame[column_name].apply(lambda x: x if pd.notnull(x) else np.random.choice(fill_values))
    


In [110]:
# cleaning cuisine column value for df_d1 dataset
remove_list_like_structure(df_d1['cuisines'],df_d1)
#replace_cuisine_value_with_most_ordered(df_d1,df_d1['cuisines'],count_occurrence_of_each_cuisine(df_d1['cuisines']))
modified_cuisine_col_value = replace_cuisine_value_with_most_ordered(df_d1,df_d1[['id','cuisines']],count_occurrence_of_each_cuisine(df_d1['cuisines']),'cuisines','id')
df_d1['cuisines'] = df_d1['id'].map(modified_cuisine_col_value).fillna(df_d1['cuisines'])

In [111]:
df_d1.isna().sum()

type                     0
id                       0
name                     0
uuid                     3
city                     0
area                     0
avgRating                0
totalRatingsString       0
cuisines                 0
costForTwoStrings        0
deliveryTime             0
minDeliveryTime          0
maxDeliveryTime          0
address                  9
locality                10
unserviceable            6
veg                      6
City                  8691
dtype: int64

In [112]:
df_d1.head()

,type,id,name,uuid,city,area,avgRating,totalRatingsString,cuisines,costForTwoStrings,deliveryTime,minDeliveryTime,maxDeliveryTime,address,locality,unserviceable,veg,City
0,F,37933,Faasos - Wraps & Rolls,6fe9caf1-02a7-4e66-83bb-1b4ff296b683,Ahmedabad,Vastrapur,4.2,500+ ratings,Indian,₹200 FOR TWO,44,44,44,SHOP NO 2 HOTEL SHAHI PALACE VASTRAPUR LAKE AH...,HOTEL SHAHI PALACE,False,False,NaN
1,F,81814,Burger King,10083576-d32d-4a0e-8a82-3236ef342a19,Ahmedabad,Ellisbridge,4.2,1000+ ratings,Fast Food,₹350 FOR TWO,33,33,33,SHOP # 5 GR FLR THIRD EYE 2PANCHAVATI CIRCLE O...,3rd Eye II Ellis Bridge CG Road,False,False,NaN
2,F,107046,Mahalaxmi Pav Bhaji,fbfe3bfa-03fd-4708-b913-06e2c9ee9639,Ahmedabad,Ellisbridge,4.3,500+ ratings,North Indian,₹200 FOR TWO,28,28,28,GF 9/10 Dev complexnear parimal charasta opp. ...,C G Road,False,True,NaN
3,F,328315,HIIT Meals,81ab4019-2445-4aef-aff2-04beccd641d9,Ahmedabad,Vishala,--,Too Few Ratings,North Indian,₹150 FOR TWO,51,51,51,C11 Rajyash Rise Near Vishala Restaurant Vasna...,Vasna,False,False,NaN
4,F,108879,Jay Jalaram Parotha House,994e73e1-7c7c-4ad9-87e7-ecfecd4e36fa,Ahmedabad,Ellisbridge,3.9,100+ ratings,North Indian,₹250 FOR TWO,29,29,29,Capital Commercial Centre Near UCO Bank Ashram...,Ashram Road,False,True,NaN


In [113]:
df_d1_new = pd.DataFrame()
swiggy_dataset_columns = ['uuid','name','city','area','avgRating','totalRatingsString','cuisines','deliveryTime','veg']
swiggy_dataset_columns_d3 = ['id','name','new_city','area','rating','rating_count','cuisine','Delivery_Time','Veg/Non-Veg']
final_dataset_columns = ['Order_Id','Restaurant_Name','City','Area','Average_Rating','Total_Rating_String','Cuisine','Delivery_Time','Veg/Non-Veg']
df_d1_new[final_dataset_columns] = df_d1[swiggy_dataset_columns].copy()
df_d1_new.head()

,Order_Id,Restaurant_Name,City,Area,Average_Rating,Total_Rating_String,Cuisine,Delivery_Time,Veg/Non-Veg
0,6fe9caf1-02a7-4e66-83bb-1b4ff296b683,Faasos - Wraps & Rolls,Ahmedabad,Vastrapur,4.2,500+ ratings,Indian,44,False
1,10083576-d32d-4a0e-8a82-3236ef342a19,Burger King,Ahmedabad,Ellisbridge,4.2,1000+ ratings,Fast Food,33,False
2,fbfe3bfa-03fd-4708-b913-06e2c9ee9639,Mahalaxmi Pav Bhaji,Ahmedabad,Ellisbridge,4.3,500+ ratings,North Indian,28,True
3,81ab4019-2445-4aef-aff2-04beccd641d9,HIIT Meals,Ahmedabad,Vishala,--,Too Few Ratings,North Indian,51,False
4,994e73e1-7c7c-4ad9-87e7-ecfecd4e36fa,Jay Jalaram Parotha House,Ahmedabad,Ellisbridge,3.9,100+ ratings,North Indian,29,True


In [114]:
df_d3 = pd.read_csv('/Users/steja/Downloads/University of Leeds/Semester 1/Programming for Data Science/Coursework 2 - GROUP PROJECT/Datasets/swiggy.csv')
df_d3.head()

,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json


In [115]:
df_d3.isna().sum()

id                0
name             86
city              0
rating           86
rating_count     86
cost            131
cuisine          99
lic_no          229
link              0
address          86
menu              0
dtype: int64

In [116]:
df_d3.dropna(inplace=True)

In [117]:
df_d3.shape

(148255, 11)

In [118]:
processed_data = pd.concat([split_city_area(city) for city in df_d3['city']], ignore_index=True)
print(processed_data)
processed_data.drop_duplicates(inplace=True)

                       city          area   new_city
0       Vastrapur,Ahmedabad     Vastrapur  Ahmedabad
1       Vastrapur,Ahmedabad     Vastrapur  Ahmedabad
2       Vastrapur,Ahmedabad     Vastrapur  Ahmedabad
3       Vastrapur,Ahmedabad     Vastrapur  Ahmedabad
4       Vastrapur,Ahmedabad     Vastrapur  Ahmedabad
...                     ...           ...        ...
100984   Dwarka Nagar,Vizag  Dwarka Nagar      Vizag
100985   Dwarka Nagar,Vizag  Dwarka Nagar      Vizag
100986   Dwarka Nagar,Vizag  Dwarka Nagar      Vizag
100987   Dwarka Nagar,Vizag  Dwarka Nagar      Vizag
100988   Dwarka Nagar,Vizag  Dwarka Nagar      Vizag

[100989 rows x 3 columns]


In [119]:
df_d3 = df_d3.merge(processed_data[['city', 'area', 'new_city']])
#pd.concat([df_d3, processed_data], ignore_index=True, sort=False)
df_d3.shape

(100989, 13)

In [120]:
df_d3.head()

,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu,area,new_city
0,40184,M.A.D By Tomato'S,"Vastrapur,Ahmedabad",4.3,100+ ratings,₹ 1200,"Indian,Chinese",10718026000103,https://www.swiggy.com/restaurants/m-a-d-by-to...,"M.A.D By Tomato'S, Ground Floor, Sumel Complex...",Menu/40184.json,Vastrapur,Ahmedabad
1,45635,Tea Post,"Vastrapur,Ahmedabad",4.0,100+ ratings,₹ 150,Fast Food,10718026000628,https://www.swiggy.com/restaurants/tea-post-va...,"Tea Post, Ground Floor, Himalaya Arcade, Opp. ...",Menu/45635.json,Vastrapur,Ahmedabad
2,483947,Shanghai Chicken Lolipops,"Vastrapur,Ahmedabad",--,Too Few Ratings,₹ 300,"Chinese,Fast Food",20721038000786,https://www.swiggy.com/restaurants/shanghai-ch...,"Shanghai Chicken Lolipops, Shop Address-GF- 17...",Menu/483947.json,Vastrapur,Ahmedabad
3,483946,Ministry Of Momos,"Vastrapur,Ahmedabad",--,Too Few Ratings,₹ 300,Chinese,20721038000786,https://www.swiggy.com/restaurants/ministry-of...,"Ministry Of Momos, Shop Address-GF- 17, MANGAL...",Menu/483946.json,Vastrapur,Ahmedabad
4,181107,Sizzling - The Cake Room,"Vastrapur,Ahmedabad",--,Too Few Ratings,₹ 350,Desserts,20718038000235,https://www.swiggy.com/restaurants/sizzling-th...,"Sizzling - The Cake Room, Beside Sizzling Pizz...",Menu/181107.json,Vastrapur,Ahmedabad


In [121]:
#replace_cuisine_value_with_most_ordered(df_d3,df_d3['cuisine'],count_occurrence_of_each_cuisine(df_d3['cuisine']))
# this block is taking lot of time to execute..try to optimize it. Note: cuisine col is not 100% cleaned as I stopped execution in between
df_d3.shape
modified_cuisine_col_value = replace_cuisine_value_with_most_ordered(df_d3,df_d3[['id','cuisine']],count_occurrence_of_each_cuisine(df_d3['cuisine']),'cuisine','id')
df_d3['cuisine'] = df_d3['id'].map(modified_cuisine_col_value).fillna(df_d3['cuisine'])

In [122]:
swiggy_dataset_columns_d3 = ['id','name','new_city','area','rating','rating_count','cuisine', 'cost']
final_dataset_columns = ['Order_Id','Restaurant_Name','City','Area','Average_Rating','Total_Rating_String','Cuisine','Total_Order_Value']
df_d3_new = pd.DataFrame()
df_d3_new[final_dataset_columns] = df_d3[swiggy_dataset_columns_d3].copy()
df_d3_new.head()

,Order_Id,Restaurant_Name,City,Area,Average_Rating,Total_Rating_String,Cuisine,Total_Order_Value
0,40184,M.A.D By Tomato'S,Ahmedabad,Vastrapur,4.3,100+ ratings,Chinese,₹ 1200
1,45635,Tea Post,Ahmedabad,Vastrapur,4.0,100+ ratings,Fast Food,₹ 150
2,483947,Shanghai Chicken Lolipops,Ahmedabad,Vastrapur,--,Too Few Ratings,Chinese,₹ 300
3,483946,Ministry Of Momos,Ahmedabad,Vastrapur,--,Too Few Ratings,Chinese,₹ 300
4,181107,Sizzling - The Cake Room,Ahmedabad,Vastrapur,--,Too Few Ratings,Desserts,₹ 350


In [123]:
df_d4 = pd.read_csv('/Users/steja/Downloads/University of Leeds/Semester 1/Programming for Data Science/Coursework 2 - GROUP PROJECT/Datasets/zomato.csv')
df_d4.shape

(51717, 17)

In [124]:
df_d4.insert(9,'City','Bengaluru')
df_d4['id'] = range(1, len(df_d4) + 1)

In [125]:
df_d4.dropna(inplace=True)
df_d4.shape

(23193, 19)

In [126]:
new_rating_dict = modify_rating_value(df_d4['rate'])
#new_rating_dict
df_d4['rate'].replace(new_rating_dict,inplace=True)

C:\Users\steja\AppData\Local\Temp\ipykernel_14676\113725405.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_d4['rate'].replace(new_rating_dict,inplace=True)


In [127]:
# For cuisine column
modified_cuisine_col_value = replace_cuisine_value_with_most_ordered(df_d4,df_d4[['id','cuisines']],count_occurrence_of_each_cuisine(df_d4['cuisines']),'cuisines','id')
df_d4['cuisines'] = df_d4['id'].map(modified_cuisine_col_value).fillna(df_d4['cuisines'])

# For Dish_liked column
modified_cuisine_col_value = replace_cuisine_value_with_most_ordered(df_d4,df_d4[['id','dish_liked']],count_occurrence_of_each_cuisine(df_d4['dish_liked']),'dish_liked','id')
df_d4['dish_liked'] = df_d4['id'].map(modified_cuisine_col_value).fillna(df_d4['dish_liked'])


In [128]:
df_d4.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,City,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),id
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,Bengaluru,Casual Dining,Pasta,North Indian,800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1,787,080 41714161,Banashankari,Bengaluru,Casual Dining,Chicken Biryani,North Indian,800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,2
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8,918,+91 9663487993,Banashankari,Bengaluru,"Cafe, Casual Dining",Hot Chocolate,Cafe,800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7,88,+91 9620009302,Banashankari,Bengaluru,Quick Bites,Masala Dosa,North Indian,300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,4
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Bengaluru,Casual Dining,Panipuri,North Indian,600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,5


In [129]:
swiggy_dataset_columns_d4 = ['name','City','location','rate','votes','cuisines','dish_liked']
final_dataset_columns = ['Restaurant_Name','City','Area','Average_Rating','Total_Rating_String','Cuisine', 'Dish_Liked']
df_d4_new = pd.DataFrame()
df_d4_new[final_dataset_columns] = df_d4[swiggy_dataset_columns_d4].copy()
df_d4_new.head()

,Restaurant_Name,City,Area,Average_Rating,Total_Rating_String,Cuisine,Dish_Liked
0,Jalsa,Bengaluru,Banashankari,4.1,775,North Indian,Pasta
1,Spice Elephant,Bengaluru,Banashankari,4.1,787,North Indian,Chicken Biryani
2,San Churro Cafe,Bengaluru,Banashankari,3.8,918,Cafe,Hot Chocolate
3,Addhuri Udupi Bhojana,Bengaluru,Banashankari,3.7,88,North Indian,Masala Dosa
4,Grand Village,Bengaluru,Basavanagudi,3.8,166,North Indian,Panipuri


In [130]:
final_dataset = pd.concat([df_d1_new, df_d3_new,df_d4_new], ignore_index=True, sort=False)
final_dataset.shape

(132873, 11)

In [131]:
final_dataset.isna().sum()

Order_Id                23196
Restaurant_Name             0
City                        0
Area                        0
Average_Rating              0
Total_Rating_String         0
Cuisine                     0
Delivery_Time          124182
Veg/Non-Veg            124188
Total_Order_Value       31884
Dish_Liked             109680
dtype: int64

In [132]:
final_dataset.sample(20)

,Order_Id,Restaurant_Name,City,Area,Average_Rating,Total_Rating_String,Cuisine,Delivery_Time,Veg/Non-Veg,Total_Order_Value,Dish_Liked
93665,450783,Malhar Chinese Family Restaurant,Pune,Hadapsar,--,Too Few Ratings,Chinese,NaN,NaN,₹ 300,NaN
98400,408283,CREAMY NUTS,Pune,Nigdi,--,Too Few Ratings,Beverages,NaN,NaN,₹ 200,NaN
69240,491963,NH1 Bowls - Highway To North,Hyderabad,Attapur,--,Too Few Ratings,North Indian,NaN,NaN,₹ 250,NaN
88390,446861,Raju's Kitchen,Mumbai,Bhayandar,--,Too Few Ratings,Chinese,NaN,NaN,₹ 350,NaN
72667,46127,Taste-Ride,Kolkata,South Kolkata,3.8,50+ ratings,Chinese,NaN,NaN,₹ 500,NaN
28708,405161,Pizza Bite,Chandigarh,South Mohali,--,Too Few Ratings,Chinese,NaN,NaN,₹ 200,NaN
24385,460459,E-Cities Foodies Corner,Bangalore,Electronic City,--,Too Few Ratings,Chinese,NaN,NaN,₹ 248,NaN
99305,139223,K K Family Restaurant,Pune,Bhosari,3.3,20+ ratings,Chinese,NaN,NaN,₹ 400,NaN
25940,384578,Sri Vasavi Chats,Bangalore,Jayanagar,4.0,20+ ratings,Indian,NaN,NaN,₹ 250,NaN
108935,425762,99 EATS,Vizag,Gajuwaka,3.8,50+ ratings,Italian,NaN,NaN,₹ 300,NaN


In [133]:
final_dataset['Average_Rating'].str.contains('--').value_counts()
# should we drop rows with 0 rating??


Average_Rating
False    67728
True     65145
Name: count, dtype: int64

In [134]:
final_dataset.shape

(132873, 11)

In [135]:
final_dataset = final_dataset[~final_dataset['Average_Rating'].str.contains('--')]

final_dataset.sample(10,ignore_index=True)

,Order_Id,Restaurant_Name,City,Area,Average_Rating,Total_Rating_String,Cuisine,Delivery_Time,Veg/Non-Veg,Total_Order_Value,Dish_Liked
0,45580,Pizza House,Hyderabad,Kondapur,3.4,100+ ratings,Fast Food,NaN,NaN,₹ 300,NaN
1,NaN,Dialogues,Bengaluru,Koramangala 4th Block,4.1,314,Continental,NaN,NaN,NaN,Pasta
2,ee74282e-6c06-435b-ad4e-e9799fc07b9e,Cafe De Italiano,Ahmedabad,Bodakdev,4,20+ ratings,Cafe,48.0,True,NaN,NaN
3,58333,Bon Bon,Coimbatore,Coi_Ganapathy,3.9,100+ ratings,Desserts,NaN,NaN,₹ 200,NaN
4,488a70bd-8f73-4bc3-b78c-efd38921cf0a,Chinese Wox,Chennai,Anna Nagar,4.6,100+ ratings,Chinese,68.0,False,NaN,NaN
5,ba44d12c-7cb2-487c-8e9a-2c2b2bee4845,KCDAS,Chennai,Thousand Lights,4.5,100+ ratings,North Indian,40.0,False,NaN,NaN
6,242957,Cafe Delhi Heights,Chandigarh,South Chd,4.1,50+ ratings,Continental,NaN,NaN,₹ 800,NaN
7,224399,Pizza Hut,Pune,Baner,3.5,100+ ratings,Fast Food,NaN,NaN,₹ 350,NaN
8,NaN,Here & Now,Bengaluru,HSR,4.6,1590,Cafe,NaN,NaN,NaN,Pasta
9,NaN,Brews N Bites,Bengaluru,Jayanagar,2.9,62,Chinese,NaN,NaN,NaN,Coffee


In [136]:
# Replace 'True' with Veg and 'False' with Non-Veg and fill the missing values
final_dataset['Veg/Non-Veg'] = final_dataset['Veg/Non-Veg'].replace({True: 'Veg', False: "Non-Veg"})
#replace_nan_values(final_dataset, 'Veg/Non-Veg')



In [137]:
final_dataset['Veg/Non-Veg'] = replace_nan_values(final_dataset,final_dataset['Veg/Non-Veg'])

In [138]:
final_dataset['Delivery_Time'] = replace_nan_values(final_dataset,final_dataset['Delivery_Time'])

In [139]:
final_dataset['Dish_Liked'] = replace_nan_values(final_dataset,final_dataset['Dish_Liked'])

In [140]:
final_dataset['Total_Order_Value'] = replace_nan_values(final_dataset,final_dataset['Total_Order_Value'])

In [141]:
final_dataset["Dish_Liked"].unique()


array(['Rolls', 'Biryani', 'Chicken Biryani', 'Burgers', 'Tea',
       'Thick Shakes', 'Mocktails', 'Hot Wings', 'Brownie',
       'Chicken Noodle', 'Garlic Bread', 'Sandwiches', 'Chaat', 'Coffee',
       'Beer', 'Pizza', 'Cocktails', 'Chicken Sandwich', 'Friendly Staff',
       'Veg Thali', 'Fries', 'Pasta', 'Mutton Biryani', 'Butter Chicken',
       'Chicken Grill', 'Paratha', 'Chocolate Cake', 'Salads', 'Waffles',
       'Sea Food', 'Prompt Service', 'Nachos', 'Filter Coffee', 'Lassi',
       'Panipuri', 'Chilli Chicken', 'Raita', 'Sangria', 'Thali',
       'Tiramisu', 'Tandoori Chicken', 'Noodles', 'Garlic Naan', 'Samosa',
       'French Fries', 'Masala Dosa', 'Fish', 'Momos', 'Roti',
       'Rooftop Ambience', 'Sandwich', 'Shawarma', 'Thukpa',
       'Vegetable Biryani', 'Benne Dosa', 'Chocolate Mousse',
       'Sweet Lassi', 'Kheer', 'Ice Cream Roll', 'Masala Chaach', 'Pie',
       'Fried Rice', 'Wood Fire Pizza', 'Chicken Chettinad',
       'Chocolate Overload', 'Nutella Waffle'

In [142]:
final_dataset.isna().sum()

Order_Id               23196
Restaurant_Name            0
City                       0
Area                       0
Average_Rating             0
Total_Rating_String        0
Cuisine                    0
Delivery_Time              0
Veg/Non-Veg                0
Total_Order_Value          0
Dish_Liked                 0
dtype: int64

In [143]:
final_dataset['Veg/Non-Veg'].value_counts(normalize=True)

Veg/Non-Veg
Non-Veg    0.714933
Veg        0.285067
Name: proportion, dtype: float64

In [144]:
final_dataset['Delivery_Time'].value_counts(normalize=True)

Delivery_Time
47.0    0.030962
49.0    0.030534
38.0    0.029249
45.0    0.029013
52.0    0.028024
          ...   
97.0    0.000325
95.0    0.000325
21.0    0.000221
20.0    0.000162
22.0    0.000118
Name: proportion, Length: 77, dtype: float64

In [145]:
final_dataset['Total_Order_Value'].value_counts(normalize=True)

Total_Order_Value
₹ 200     0.210120
₹ 300     0.193273
₹ 250     0.131674
₹ 400     0.109689
₹ 150     0.069779
            ...   
₹ 284     0.000015
₹ 2700    0.000015
₹ 104     0.000015
₹ 390     0.000015
₹ 155     0.000015
Name: proportion, Length: 183, dtype: float64

In [146]:
final_dataset['Dish_Liked'].value_counts(normalize=True)

Dish_Liked
Pasta                0.143441
Cocktails            0.082802
Burgers              0.078092
Biryani              0.070237
Coffee               0.049994
                       ...   
Aloo Tikki Chat      0.000030
Fruit Cake           0.000030
Strawberry Cake      0.000030
Chocolava            0.000015
Paneer Manchurian    0.000015
Name: proportion, Length: 305, dtype: float64

In [147]:
final_dataset.to_csv('D1_D3_D4_new.csv', index=False)